# Quick tutorial. 
## Finding eigenvalues of a density matrix using Newton Identities and the Faddeev LeVerrier algorithm.
In this document we show how to find the eigenvalues of a density matrix using the Faddeev Leverrier (F-L) algorithm. We compute an array of traces of the different powers of the density matrix: $ \{ \text{Tr} A^i \}\; i=1,...,d$ where $d$ is the dimension of the system. This array is used by the F-L algorithm to compute the coefficients of the characteristic polynomial. The polynomial can be solved with any method suited for that task, like the ones found in numpy

**Disclaimer**: The algorithms are direct implementations of the pseucode available at Wikipedia. There might be errors. We have checked  solutions finding the eigenvalues with numpy and comparing the results, but proper testing of the functions is required. 

## Required packages

In [53]:
import numpy as np
import qutip as qu
from numba import jit

## Generate a density matrix of two qubits (dimension = 4)

In [54]:
seed = 2
n = 2
d = 2**n
rng = np.random.default_rng(seed)
rho = qu.rand_dm(d)[:]
# rho = rho*100

## Function that computes the traces of the different powers of the matrix

In [55]:
def get_traces(rho):
    traces = []
    for i in range(rho.shape[0]):
        traces.append(np.trace( np.linalg.matrix_power(rho, i+1)))
    return np.array(traces)

In [56]:
traces = get_traces(rho)
print(f'Traces: {traces}')
print(f'Dimension of vector of traces: {traces.shape}')
print('Makes sense that the first trace is one bc we are dealing with a density matrix')

Traces: [1.        +0.00000000e+00j 0.43721059+0.00000000e+00j
 0.22992778-8.67361738e-19j 0.12899769+0.00000000e+00j]
Dimension of vector of traces: (4,)
Makes sense that the first trace is one bc we are dealing with a density matrix


## Implementation of the F-L algorithm
We use '@jit(nopython=True)' decorator from Numba to compile the function and make it faster when used with bigger matrices.

In [57]:
@jit(nopython=True)
def c_n_m(n, m, vec_of_traces):
    """Auxiliary function used in the recursive part of the Faddeev LeVerrier algorithm.
    Ref: https://en.wikipedia.org/wiki/Faddeev%E2%80%93LeVerrier_algorithm
    """
    suma = 0
    if (m==0):
        return 1.

    for k in range(1, m+1):
        suma = suma + c_n_m(n, m-k, vec_of_traces) * vec_of_traces[k-1]
    return suma*(-1/m)   


@jit(nopython=True) 
def compute_coefs(n, vec_of_traces):
    """Compute coefficients of characteristic polynomial using
    the Faddeev LeVerrier algorithm.

    Refs: https://en.wikipedia.org/wiki/Newton%27s_identities
          https://en.wikipedia.org/wiki/Faddeev%E2%80%93LeVerrier_algorithm


    Args:
        n (int): Degree of the polynomial. In this case is the dimension of the density matrix
        vec_of_traces (np.array[float]): Array with the traces of A^i for i=1,..., n

    Returns:
        np.array[float]: Array with the coefficients of the characteristic polynomial.
    """
    c_list = [1.]
    for m in range(1, n+1):
        c_list.append(c_n_m(n, m, vec_of_traces) )
    
    return np.array(c_list)   

# Checking of the algorithm

Compute the coefficients of the polynomial.

In [58]:
coefs = compute_coefs(d, get_traces(rho))

Generate a numpy polynomial with the coefficients. The array is used in reverse because of the way the numpy function works.
We find the roots of the polynomial with the method 'roots()' and print the array of the eigenvalues. all of them are real and positive.


In [59]:
poly = np.polynomial.polynomial.Polynomial(coefs[::-1] )
np_roots = poly.roots()
print(np.sort(np_roots))

[0.05024531-7.66118421e-17j 0.07838776+8.22564533e-17j
 0.2793166 -3.66375758e-18j 0.59205033-1.29189465e-17j]


Let's compute the eigenvalues of the density matrices using the function 'np.linal.eig' and check that they are the same.

In [60]:
print(np.sort(np.real(np.linalg.eig(rho)[0])))

[0.05024531 0.07838776 0.2793166  0.59205033]


Let's print the polynomial. We see the difference in size between the coefficients. The constant term is very small ~ order(1E-4) while the term of 4th order is 1. This will create instabilities when solving the algorithm. Specially in bigger dimensions.

In [61]:
print(poly)

(0.0006513271093665438-2.8912057932946783e-19j) -
(0.024703965283386788-2.8912057932946783e-19j)·x¹ +
(0.28139470627328345-0j)·x² - (1-0j)·x³ + (1+0j)·x⁴
